In [1]:
import os
import sys

sys.path.insert(0, '../')

In [2]:
import pandas as pd
import numpy as np

from src.models.model_common import select_features

/Users/sreejithkrishnanr/Tools/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
train_data = pd.read_hdf('../data/processed/train_900000000000.hd5', 'data', parse_dates=[1])

In [31]:
site_data = train_data.loc[train_data['SiteId'] == 25, :]
site_data.head()

,ForecastId,Timestamp,obs_id,SiteId,Value,Frequency,DayOfMonth,DayOfWeek,DayOfYear,DaysInMonth,...,ConsumptionDailyMeanPerSurfaceArea,ConsumptionDailyMeanPerTemperatureDiff,ConsumptionWeeklyMeanPerSurfaceArea,ConsumptionWeeklyMeanPerTemperatureDiff,ConsumptionHalfDayMeanPerSurfaceArea,ConsumptionHalfDayMeanPerTemperatureDiff,ConsumptionQuarterDayMeanPerSurfaceArea,ConsumptionQuarterDayMeanPerTemperatureDiff,ConsumptionHourlyMeanPerSurfaceArea,ConsumptionHourlyMeanPerTemperatureDiff
617924,747,2014-12-19 15:30:00,4674101,25,46974.397954,9.000000e+11,19,4,353,31,...,10.895833,265.837194,11.90625,290.489400,11.222222,273.800450,13.777778,336.151047,20.333333,496.093884
617925,747,2014-12-19 15:45:00,5853656,25,45506.448018,9.000000e+11,19,4,353,31,...,10.895833,239.918068,11.90625,262.166684,11.222222,247.104906,13.777778,303.376320,20.333333,447.724730
617926,747,2014-12-19 16:00:00,154522,25,44038.498081,9.000000e+11,19,4,353,31,...,10.895833,245.634848,11.90625,268.413605,11.222222,252.992934,13.777778,310.605187,20.333333,458.393139
617927,747,2014-12-19 16:15:00,5974685,25,42570.548145,9.000000e+11,19,4,353,31,...,10.895833,239.918068,11.90625,262.166684,11.222222,247.104906,13.777778,303.376320,20.333333,447.724730
617928,747,2014-12-19 16:30:00,3612810,25,39634.648273,9.000000e+11,19,4,353,31,...,10.895833,239.918068,11.90625,262.166684,11.222222,247.104906,13.777778,303.376320,20.333333,447.724730


In [32]:
x, y, groups, ts = select_features(site_data, '900s')

In [33]:
from xgboost import XGBRegressor

regressor = XGBRegressor()

In [57]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(regressor, {
    'learning_rate': [0.13],
    'max_depth': [9],
    'n_estimators': [50],
    'colsample_bytree': [1],
    'colsample_bylevel': [0.6],
    'gamma': [0],
    'reg_alpha': [1.2],
    'reg_lambda': [1.4, 1.5, 1.6]
}, n_jobs=8, cv=8, scoring='neg_mean_squared_error')

gs.fit(x, y)

GridSearchCV(cv=8, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'learning_rate': [0.13], 'max_depth': [9], 'n_estimators': [50], 'colsample_bytree': [1], 'colsample_bylevel': [0.6], 'gamma': [0], 'reg_alpha': [1.2], 'reg_lambda': [1.4, 1.5, 1.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [58]:
gs.best_params_

{'colsample_bylevel': 0.6,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.13,
 'max_depth': 9,
 'n_estimators': 50,
 'reg_alpha': 1.2,
 'reg_lambda': 1.4}

In [30]:
gs.best_score_

-18351.728834226025